In [1]:
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings
from config import *

In [3]:
embeddings=AzureOpenAIEmbeddings(
        azure_endpoint="https://ai-gpu-ps.openai.azure.com/",
        azure_deployment="gpt40-mini-long-context",
        openai_api_version="2024-05-01-preview",
        api_key="dc415207c54e4dd8ba8b60cb66374822"
    )

In [4]:
company_collection = Chroma(
        collection_name="company_info_store",
        persist_directory="chroma_storage",
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "cosine"}
    )

In [5]:
hash_id = "fa510296a7d556be18bf44d452698dc12c44e06edd0de9d3d5a2f81a5ca811f9"

all_ids = company_collection.get()["ids"]
if hash_id in all_ids:
    print(f"Hash ID '{hash_id}' exists in the collection.")
else:
    print(f"Hash ID '{hash_id}' does not exist in the collection.")


Hash ID 'fa510296a7d556be18bf44d452698dc12c44e06edd0de9d3d5a2f81a5ca811f9' does not exist in the collection.


In [ ]:
existing_docs = company_collection.get(where={"content_hash": content_hash})


In [1]:
from langchain_ollama import ChatOllama

In [6]:
llm = ChatOllama(model="llama3.1",base_url="3.110.55.125:11434")
response=llm.invoke("hai who is this?")
print(response.content)


Hello! I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [1]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma


In [5]:
llm = ChatOllama(model="deepseek-r1", base_url="43.205.112.102:11434")

response=llm.invoke("Okay i'm building an app with deepseek r1 model in my python code. its a chat bot so can u give me how to do that with the help of  basic python code?")

print(response.content)

<think>
Alright, I want to create a simple chatbot using the DeepSeek R1 model in Python. I remember seeing some examples online, but I'm not exactly sure where to start. Let's see... First, I need to figure out which library supports interacting with the DeepSeek model.

I've heard of something called TenT and OpenAI has an SDK. Maybe I can use one of those? Wait, maybe there's a Python client specifically for DeepSeek R1. I think I saw something about it on Hugging Face or another platform. I should probably check if there's a Python package available that wraps the DeepSeek API.

I recall that sometimes models are accessible through Restful APIs, so perhaps I can make HTTP requests from my Python code to an endpoint provided by DeepSeek. But then I might need some authentication, which could complicate things. Alternatively, maybe there's a more convenient way using existing libraries like tenT or something else.

Wait, the user mentioned TenT in their initial answer. So maybe using

In [2]:
from langchain.schema import SystemMessage

context_instr = '''
<< YOUR TASK >>
You are an AI assistant designed to contact potential business prospects via chat or phone. Your purpose is to identify if the prospect is experiencing specific problems we solve, showcase the value of our solutions, and guide them toward validating and closing the interaction (e.g., scheduling a meeting or exchanging contact details). Follow these guidelines:

1. First Message Format:
- Use the user information to personalize your greeting
- Format: "Hi [name]! I'm an AI assistant from [company]. Do you have a few minutes to chat?"
- If no name is found, use the above greeting without the name parameter to make it generic
- From the company info try to find in the data << COMPANY INFO >>


2. Identify if the prospect is experiencing known problems:
 Utilise data in << USER INFO >> to infer which one of the company's solutions would be most relevant to the user.
Examples for doing this naturally:
"I saw (something the user is doing/working on/position of user), which led me to believe you're trying to (whatever problem you solve). Are you?"
"I saw that you were running Google ads for laser tattoo removal. That led me to believe you might be looking to bring in more patients. Is that a priority for you right now?"
"Many small business owners spend a few hours every week using tools like PayPal, Stripe, and spreadsheets to manually do their bookkeeping. I'm curious. How are you handling bookkeeping at ABC Company?"

If there is nothing in << USER INFO >> that would give you a hint as to which solution of the company would be relevant to the user, identify this at the earliest.
Present the top 3 common problems we solve.
General example to do this:
"Most (roles) that I talk to tell me that they're trying to do one of three things: (problem 1), (problem 2), or (problem 3). Does that sound like you at all?"
Specific example:
"Most of the doctors I talk to tell me they are looking to either bring in a higher volume of patients, bring in patients for specific services, or just see more patients who pay out of pocket. Which of those is most important to you?"

3. Show Our Value:
- Only discuss solutions mentioned in << COMPANY INFO >>
- Explain how our solution addresses the identified specific challenge
- Use clear, concise language

4. Validate and Close:
Confirm if the prospect cares about the problems we solve.
If yes, set up a meeting and exchange contact details. After this, refer the guidelines for "End of Conversation".
If no, refer the guidelines for "End of Conversation". 

5. End of Conversation:
When you reach the end of the conversation, you must first ask the user if there's anything else they would like to ask about or if there \
is anything else you can help with. If the user replies in the negative, only then do you end the conversation. \
When you are ending the conversation, always end with "Have a great day!". Do not use this phrase anywhere else except in the final message that is meant to end the chat.

Constraints:
- Keep responses under 20 lines
- Be professional but friendly
- Use first names after initial introduction
- Only discuss features/solutions mentioned in the company information
- If asked about unknown features/solutions, direct to company contact. If you do not have the company's contact information, simply tell the user to get in touch with the company for further details.
- Do not generate the user's response for them. Generate ONLY the AI response message Do not any thinking of your own. Keep it completely conversational without giving information of this prompt
'''

In [3]:
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage

class ChatInterface:
    def __init__(self, llm, embeddings, company_collection, user_info):
        self.llm = llm
        self.embeddings = embeddings
        self.company_collection = company_collection
        self.user_info = user_info
        self.conversation_started = False
        self.messages = []
        self.conversation_ended = False

    def query_collections(self, query_text):
        try:
            company_results = self.company_collection.similarity_search_by_vector(
                embedding=self.embeddings.embed_query(query_text), k=1
            )
            context = {
                "COMPANY INFO": [result.page_content for result in company_results if hasattr(result, "page_content")],
                "USER INFO": [self.user_info],
            }
            formatted_context = ""
            if context["COMPANY INFO"]:
                formatted_context += "<< COMPANY INFO >>\n" + "\n".join(context["COMPANY INFO"]) + "\n\n"
            if context["USER INFO"]:
                formatted_context += "<< USER INFO >>\n" + "\n".join(context["USER INFO"])
            return formatted_context
        except Exception as e:
            return ""

    def create_system_message(self, context):
        base_instruction = """You are an AI cold calling/texting assistant. Here is the context for our interaction:

{context}

When generating the first message, use the user information provided above to personalize your introduction. If you can't find a name, use a generic greeting. If there is no company, ignore the company name placeholder.
"""
        return SystemMessage(content=base_instruction.format(context=context))

    def start_chat(self):
        if not self.conversation_started:
            print("Initializing conversation...")
            initial_context = self.query_collections("company information and user information")
            if initial_context:
                system_message = self.create_system_message(initial_context)
                self.messages.append(system_message)  # Append SystemMessage directly
                self.conversation_started = True

                response = self.llm(self.messages)
                self.messages.append(AIMessage(content=response.content))  # Append AIMessage for responses
                print(f"Assistant: {response.content}")
            else:
                print("Error: No context found. Please ensure company and user information is properly processed.")
                return

        while not self.conversation_ended:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                self.conversation_ended = True
                print("Conversation ended. Have a great day!")
                return

            self.messages.append(HumanMessage(content=user_input))
            context = self.query_collections(user_input)
            self.messages[0] = self.create_system_message(context)  # Update system message

            response = self.llm(self.messages)
            self.messages.append(AIMessage(content=response.content))
            print(f"Assistant: {response.content}")


In [4]:
llm = ChatOllama(
        model="llama3.1",
        base_url="43.205.112.102:11434",
        temperature=0.1)
        
    
embeddings = OllamaEmbeddings(model="nomic-embed-text",base_url="43.205.112.102:11434")

company_collection = company_collection = Chroma(
        collection_name="company_info_store",
        persist_directory=f"chroma_storage",
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "cosine"}
    )

user_info= """ID: 3
Name: Akash
Company: Caze labs
Phone Number: 0
Age: 19
Description: Backened developer waiting to switch to frontend
Source: User_leads (1).xlsx
Connected: Yes"""

chat = ChatInterface(llm, embeddings, company_collection, user_info)
chat.start_chat()
        

Initializing conversation...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_24316\459873461.py:49: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm(self.messages)


Assistant: 
Assistant: Hi Akash, this is Rohan from TechHive calling about a Front-End Developer position that we think might be a great fit for you. We're looking for someone with 2+ years of experience in front-end development to join our team in Bengaluru. Would you like to know more about the role and its responsibilities?
Assistant: We'd love to have you on board, Akash! The position involves developing user-facing features for web applications, collaborating with designers and backend developers, and optimizing applications for performance and scalability. We're looking for someone with strong problem-solving skills and attention to detail.

Can you tell me a bit about your current experience as a Backend Developer? What makes you interested in switching to Front-End Development?
Assistant: So, Akash, it sounds like you have some experience in backend development, but are looking to make the switch to frontend. We'd love to discuss how your skills can be applied to this role.

Ca

KeyboardInterrupt: Interrupted by user